# Introduction to Quantitative Finance and Financial Risk
## Theodoros Anagnos p3352323

# Triangular arbitrage

In [1]:
import pandas as pd
import itertools
import time

# Triangular Arbitrage Analysis

Here we to explore all possibilities of triangular arbitrage, produce the corresponding trades, and record the overall results iteratively for all currencies of the dataset.

## How It Works

### 1. Load and Prepare Data
It starts by loading the Excel file containing currency exchange rates into a pandas DataFrame. It fills any NaN values with 0 to simplify calculations.

In [2]:
def load_and_prepare_data(file_path):
    """
    Load the Excel file into a DataFrame and fill NaN values with 0.
    """
    df = pd.read_excel(file_path)
    return df.fillna(0)

### 2. Create Bid and Ask Dictionaries

Next, we restructure the DataFrame into bid and ask dictionaries. The bid dictionary contains the original exchange rates, while the ask dictionary contains the reciprocal of these rates

In [3]:
def create_bid_ask_dict(df):
    """
    Restructure the DataFrame into bid and ask dictionaries.
    """
    bid = {}
    for row in df.index:
        currency = df.loc[row, 'Unnamed: 0']
        for col in df.columns[1:]:
            value = df.loc[row, col]
            if value != 0:
                key = f"{currency}|{col}"
                bid[key] = value

    ask = {f"{k.split('|')[1]}|{k.split('|')[0]}": 1/v for k, v in bid.items()}
    return bid, ask

### 3. Generate Triangular Arbitrage Opportunities

It then generates all possible triangular arbitrage opportunities by considering all combinations of three trades. It calculates the potential value after performing these trades in sequence.

In [4]:
def generate_combinations(bid, ask):
    """
    Generate all possible combinations from bid and ask dictionaries.
    """
    combinations = list(itertools.product(bid.keys(), repeat=3))
    paths = {}

    for combo in combinations:
        tr1 = bid[combo[0]]
        tr2 = ask[combo[1]]
        tr3 = bid[combo[2]]

        if combo[0].split("|")[0] == combo[1].split("|")[0] and \
           combo[0].split("|")[1] == combo[2].split("|")[0] and \
           combo[1].split("|")[1] == combo[2].split("|")[1]:

            paths[combo] = tr1 * tr3 / tr2

    return paths

def filter_paths_by_currency(paths, currency):
    """
    Filter paths by the given starting currency.
    """
    filtered_paths = {k: v for k, v in paths.items() if k[0].split("|")[1] == currency}
    return filtered_paths

### 4. Print Triangular Arbitrage Opportunities

For each currency, it filters and prints the profitable triangular arbitrage opportunities. It distinguishes between profitable and unfavorable trades.

In [5]:
def print_filtered_trades(sorted_paths, currency, limit=20):
    """
    Print filtered trades for a specific currency, distinguishing between profitable and unfavorable ones.
    """
    count = 0
    flag = True

    print(f"\nBest Trades for {currency}:")
    for key, value in sorted_paths:
        if count < limit:
            count += 1
            if value > 1:
                print(f"{key}: {value}")
            else:
                if flag:
                    print("--------------Unfavorable Trades------------------")
                    flag = False
                print(f"{key}: {value}")

In [6]:
def main():
    # Load and prepare data
    df = load_and_prepare_data('./Quotation Matrix.xlsx')

    # Create bid and ask dictionaries
    bid, ask = create_bid_ask_dict(df)

    # Generate all possible combinations and calculate their values
    paths = generate_combinations(bid, ask)

    # Iterate over all currencies
    options = ["USD", "EUR", "JPY", "GBP", "CHF", "CAD", "AUD"]
    for currency in options:
        # Filter paths and sort by values in descending order
        filtered_paths = filter_paths_by_currency(paths, currency)
        sorted_paths = sorted(filtered_paths.items(), key=lambda x: x[1], reverse=True)

        # Print filtered trades
        print_filtered_trades(sorted_paths, currency)

if __name__ == "__main__":
    start_time = time.time()
    main()
    print(f"--- Execution time: {time.time() - start_time} seconds ---")


Best Trades for USD:
('AUD|USD', 'AUD|JPY', 'USD|JPY'): 1.0022087286317982
('JPY|USD', 'JPY|GBP', 'USD|GBP'): 1.0017824345459168
('CHF|USD', 'CHF|JPY', 'USD|JPY'): 1.0007557937565956
--------------Unfavorable Trades------------------
('CAD|USD', 'CAD|JPY', 'USD|JPY'): 0.9999911904673027
('GBP|USD', 'GBP|CHF', 'USD|CHF'): 0.999636086368835
('EUR|USD', 'EUR|CHF', 'USD|CHF'): 0.9996150852555385
('EUR|USD', 'EUR|JPY', 'USD|JPY'): 0.9995486773579649
('EUR|USD', 'EUR|CAD', 'USD|CAD'): 0.9994745320580963
('CHF|USD', 'CHF|CAD', 'USD|CAD'): 0.99945494220348
('GBP|USD', 'GBP|CAD', 'USD|CAD'): 0.9994035094888079
('EUR|USD', 'EUR|GBP', 'USD|GBP'): 0.9987484773041231
('AUD|USD', 'AUD|CAD', 'USD|CAD'): 0.9986823067768622
('EUR|USD', 'EUR|AUD', 'USD|AUD'): 0.9986553631989169
('CHF|USD', 'CHF|AUD', 'USD|AUD'): 0.9986546808744285
('GBP|USD', 'GBP|AUD', 'USD|AUD'): 0.9986298201810069
('CAD|USD', 'CAD|AUD', 'USD|AUD'): 0.9986209178532689
('CAD|USD', 'CAD|CHF', 'USD|CHF'): 0.9985470225634541
('AUD|USD', 

## Results Summary

#### Profitable Trades
1. **USD:**
   - Profitable trades involved converting USD to other currencies such as AUD, JPY, GBP, and CHF, and then back to USD. The profits ranged from 0.08% to 0.22%.
   
2. **EUR:**
   - Profitable trades involved converting EUR to JPY, GBP, AUD, and back to EUR. The profits ranged from 0.14% to 0.23%.
   
3. **JPY:**
   - JPY had several profitable trades involving GBP, AUD, CHF, CAD, and USD. The profits ranged from 0.08% to 0.58%, with the highest profit involving GBP.
   
4. **GBP:**
   - GBP's profitable trades involved AUD, JPY, CHF, CAD, and EUR, with profits ranging from 0.36% to 0.58%, particularly high when involving AUD and JPY.
   
5. **CHF:**
   - Profitable trades for CHF included pairs with JPY, GBP, and USD. The profits ranged from 0.08% to 0.44%, with JPY and GBP yielding the highest profits.
   
6. **CAD:**
   - CAD's profitable trades involved JPY, GBP, and AUD. Profits ranged from 0.13% to 0.36%, with notable returns from JPY and GBP trades.
   
7. **AUD:**
   - Profitable trades for AUD included JPY, GBP, and USD pairs. The profits ranged from 0.13% to 0.58%, with JPY and GBP providing the highest returns.

#### Unfavorable Trades
1. **USD:**
   - Unfavorable trades involved pairs like CAD, GBP, EUR, and CHF. These trades resulted in small losses, typically under 0.1%.
   
2. **EUR:**
   - Unfavorable trades included pairs with CHF, CAD, GBP, and USD. Losses were generally minimal, around 0.08% to 0.09%.
   
3. **JPY:**
   - Unfavorable trades for JPY involved USD, CAD, EUR, and CHF, with losses up to 0.1%.
   
4. **GBP:**
   - Unfavorable trades included pairs with CHF, CAD, EUR, and USD. Losses were typically under 0.1%, similar to other currencies.
   
5. **CHF:**
   - Unfavorable trades for CHF involved EUR, CAD, USD, and GBP, with losses around 0.08% to 0.1%.
   
6. **CAD:**
   - Unfavorable trades included pairs with USD, EUR, CHF, and GBP. Losses were generally under 0.1%.
   
7. **AUD:**
   - Unfavorable trades for AUD involved EUR, GBP, CHF, and USD. Losses were typically under 0.1%.

### Key Insights
- **Highest Profits:** The highest profits were generally found in trades involving GBP and JPY, particularly with JPY yielding profits up to 0.58%.
- **Common Patterns:** Profitable trades often involved the same currencies (e.g., JPY and GBP) across different base currencies.
- **Low-Risk Trades:** Unfavorable trades typically resulted in minimal losses, indicating that the triangular arbitrage opportunities are relatively low-risk with small profit margins.

### Execution Time
The analysis was completed efficiently in approximately 1.25 seconds, demonstrating the program's capability to handle extensive calculations quickly.

### Conclusion
The generalized results show that triangular arbitrage opportunities exist across various currency pairs, with certain trades yielding consistent profits. The analysis highlights the importance of carefully selecting trade pairs to maximize profits while minimizing potential losses. This information can be leveraged to develop effective currency trading strategies.

# One way arbitrage

In [9]:
def generate_one_way_arbitrage(bid, ask):
    """
    Generate all possible one-way arbitrage opportunities.
    """
    arbitrage_opportunities = {}

    for key, bid_value in bid.items():
        currency1, currency2 = key.split('|')
        ask_key = f"{currency2}|{currency1}"
        if ask_key in ask:
            ask_value = ask[ask_key]
            profit = bid_value - ask_value
            if profit > 0:
                arbitrage_opportunities[(key, ask_key)] = profit

    return arbitrage_opportunities

def print_arbitrage_opportunities(arbitrage_opportunities, currency, limit=20):
    """
    Print one-way arbitrage opportunities for a specific currency.
    """
    count = 0
    flag = True

    print(f"\nArbitrage Opportunities for {currency}:")
    for (buy, sell), profit in arbitrage_opportunities.items():
        if count < limit:
            count += 1
            if profit > 0:
                print(f"Buy: {buy}, Sell: {sell}, Profit: {profit}")
            else:
                if flag:
                    print("--------------Unfavorable Trades------------------")
                    flag = False
                print(f"Buy: {buy}, Sell: {sell}, Profit: {profit}")

def main():
    df = load_and_prepare_data('./Quotation Matrix.xlsx')
    
    # Create bid and ask dictionaries
    bid, ask = create_bid_ask_dict(df)

    # Generate all possible one-way arbitrage opportunities
    arbitrage_opportunities = generate_one_way_arbitrage(bid, ask)

    # Iterate over all currencies
    options = ["USD", "EUR", "JPY", "GBP", "CHF", "CAD", "AUD"]
    for currency in options:
        # Filter opportunities for the specific currency
        filtered_opportunities = {
            k: v for k, v in arbitrage_opportunities.items() if k[0].split('|')[0] == currency or k[1].split('|')[1] == currency
        }
        # Print filtered arbitrage opportunities
        print_arbitrage_opportunities(filtered_opportunities, currency)

if __name__ == "__main__":
    start_time = time.time()
    main()
    print(f"--- Execution time: {time.time() - start_time} seconds ---")



Arbitrage Opportunities for USD:
Buy: USD|EUR, Sell: EUR|USD, Profit: 0.6856760852570252
Buy: USD|GBP, Sell: GBP|USD, Profit: 1.1817747955656297

Arbitrage Opportunities for EUR:
Buy: EUR|GBP, Sell: GBP|EUR, Profit: 0.45092480436596927

Arbitrage Opportunities for JPY:
Buy: JPY|USD, Sell: USD|JPY, Profit: 107.85072872241795
Buy: JPY|EUR, Sell: EUR|JPY, Profit: 151.21338711810606
Buy: JPY|GBP, Sell: GBP|JPY, Profit: 188.3913423575575
Buy: JPY|CHF, Sell: CHF|JPY, Profit: 95.12293689480593
Buy: JPY|CAD, Sell: CAD|JPY, Profit: 100.9093647538036
Buy: JPY|AUD, Sell: AUD|JPY, Profit: 86.92763161234797

Arbitrage Opportunities for GBP:

Arbitrage Opportunities for CHF:
Buy: CHF|USD, Sell: USD|CHF, Profit: 0.2542979563072586
Buy: CHF|EUR, Sell: EUR|CHF, Profit: 0.9631619541313226
Buy: CHF|GBP, Sell: GBP|CHF, Profit: 1.489991967871486
Buy: CHF|CAD, Sell: CAD|CHF, Profit: 0.1175686146507422

Arbitrage Opportunities for CAD:
Buy: CAD|USD, Sell: USD|CAD, Profit: 0.13410875339006811
Buy: CAD|EUR, S

## Results Summary

#### Arbitrage Opportunities for USD:
1. **Profitable Trades:**
   - **USD to EUR and back to USD:** Profit of 0.686%
   - **USD to GBP and back to USD:** Profit of 1.182%

#### Arbitrage Opportunities for EUR:
1. **Profitable Trades:**
   - **EUR to GBP and back to EUR:** Profit of 0.451%

#### Arbitrage Opportunities for JPY:
1. **Profitable Trades:**
   - **JPY to USD and back to JPY:** Profit of 107.851%
   - **JPY to EUR and back to JPY:** Profit of 151.213%
   - **JPY to GBP and back to JPY:** Profit of 188.391%
   - **JPY to CHF and back to JPY:** Profit of 95.123%
   - **JPY to CAD and back to JPY:** Profit of 100.909%
   - **JPY to AUD and back to JPY:** Profit of 86.928%

#### Arbitrage Opportunities for GBP:
1. **Profitable Trades:**
   - No profitable trades were found for GBP in this analysis.

#### Arbitrage Opportunities for CHF:
1. **Profitable Trades:**
   - **CHF to USD and back to CHF:** Profit of 0.254%
   - **CHF to EUR and back to CHF:** Profit of 0.963%
   - **CHF to GBP and back to CHF:** Profit of 1.490%
   - **CHF to CAD and back to CHF:** Profit of 0.118%

#### Arbitrage Opportunities for CAD:
1. **Profitable Trades:**
   - **CAD to USD and back to CAD:** Profit of 0.134%
   - **CAD to EUR and back to CAD:** Profit of 0.832%
   - **CAD to GBP and back to CAD:** Profit of 1.343%

#### Arbitrage Opportunities for AUD:
1. **Profitable Trades:**
   - **AUD to USD and back to AUD:** Profit of 0.440%
   - **AUD to EUR and back to AUD:** Profit of 1.171%
   - **AUD to GBP and back to AUD:** Profit of 1.725%
   - **AUD to CHF and back to AUD:** Profit of 0.183%
   - **AUD to CAD and back to AUD:** Profit of 0.304%

### Key Insights
- **Highest Profits:** The highest profits were generally found in trades involving JPY. The JPY to GBP pair yielded the highest profit at 188.391%.
- **Common Patterns:** Profitable trades often involve converting to a currency with a strong return rate and then converting back. JPY demonstrated significant profitability across multiple pairs.
- **Low-Risk Trades:** Most profitable trades resulted in substantial profits, indicating potential high returns with relatively low risk in these particular pairs.

### Execution Time
The entire analysis was completed efficiently in approximately 0.011 seconds, demonstrating the program's capability to handle extensive calculations quickly.

### Conclusion
The generalized results show that one-way arbitrage opportunities exist across various currency pairs, with significant profits observed particularly in JPY pairs. This analysis helps in making informed decisions for currency trading strategies by identifying the most profitable arbitrage opportunities.